In [ ]:
import ast
import json
from bs4 import BeautifulSoup
from transformers import pipeline
summarizer = pipeline("summarization")
import re
import pandas as pd
import numpy as np
import os
os.chdir(r'..\data')
from keybert import KeyBERT
kw_model = KeyBERT()
from tqdm import tqdm
from joblib import Parallel, delayed
import multiprocessing
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Opening JSON file
f = open('articles.json', "r", encoding='utf-8')
data = json.load(f)

In [ ]:
data['doc1'].keys()

In [ ]:
docid_list = [docid for docid in data.keys()]
docid_list

In [ ]:
doc_attributes = data[docid_list[0]].keys()
doc_attributes = list(doc_attributes)
doc_attributes

In [ ]:
def get_document_text(docid):
    title = data[docid]['title']
    excerpt = data[docid]['excerpt']
    body = data[docid]['body']
    combined_text = title +'\n'+excerpt+'\n'+body
    soup_combined_text = BeautifulSoup(combined_text)
    clean_combined_text = soup_combined_text.get_text()
    return clean_combined_text

def get_top_n_phrases(doctext, n, colls):
    keywords = kw_model.extract_keywords(doctext, keyphrase_ngram_range=(2, 3), stop_words='english', top_n=n)
    keywords_df = pd.DataFrame(keywords, columns=colls)
    summarize_text = summarizer(doctext, max_length=130, min_length=30, do_sample=False)
    summarize_text = summarize_text[0]['summary_text']
    return keywords_df, summarize_text

In [ ]:
colls = ['phrase', 'importance']
clean_combined_text = get_document_text('doc1')
result_df, summaries_text = get_top_n_phrases(clean_combined_text, 5, colls)
display(result_df)
summaries_text

In [ ]:
clean_combined_text = get_document_text('doc2')
result_df, summaries_text = get_top_n_phrases(clean_combined_text, 5, colls)
display(result_df)
summaries_text

In [ ]:
clean_combined_text = get_document_text('doc3')
result_df, summaries_text = get_top_n_phrases(clean_combined_text, 5, colls)
display(result_df)
summaries_text

In [ ]:
clean_combined_text = get_document_text('doc4')
result_df, summaries_text = get_top_n_phrases(clean_combined_text, 5, colls)
display(result_df)
summaries_text

In [ ]:
clean_combined_text = get_document_text('doc5')
result_df, summaries_text = get_top_n_phrases(clean_combined_text, 5, colls)
display(result_df)
summaries_text

In [ ]:
# Opening JSON file
f = open('newsSample.json', "r", encoding='utf-8')
data = json.load(f, strict=False)

In [ ]:
len(data)

In [ ]:
data[45]

In [ ]:
doc_df = pd.DataFrame(data, columns=['json_doc'])
doc_df.head(2)

In [ ]:
doc_index=['doc_'+str(idx) for idx, json_doc in enumerate(data)]
len(doc_index), len(data)

In [ ]:
doc_df = pd.DataFrame({'doc_index':doc_index,'doc':data})
def get_document_text(jdoc):
    title = jdoc['title']
    excerpt = jdoc['excerpt']
    body = jdoc['body']
    combined_text = title +'\n'+excerpt+'\n'+body
    soup_combined_text = BeautifulSoup(combined_text)
    clean_combined_text = soup_combined_text.get_text()
    return clean_combined_text

def get_top_n_phrases(jdoc, n, colls):
    doctext = get_document_text(jdoc)
    keywords = kw_model.extract_keywords(doctext, keyphrase_ngram_range=(2, 3), stop_words='english', top_n=n)
    keywords_df = pd.DataFrame(keywords, columns=colls)
    summarize_text = summarizer(doctext, max_length=130, min_length=30, do_sample=False)
    summarize_text = summarize_text[0]['summary_text']
    return keywords_df, summarize_text

doc_df['extraction'] = doc_df['doc'].process_apply(get_top_n_phrases, n=5, colls=colls)

In [ ]:
process_doc_ls = Parallel(n_jobs=4, backend="multiprocessing")(delayed(get_top_n_phrases)(jsondoc, 5, colls) for jsondoc in tqdm(doc_df['doc'].tolist(), total=doc_df.shape[0]))